In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline



In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten
from sklearn.preprocessing import MinMaxScaler

In [ ]:
df = pd.read_csv('Poblacion.csv', parse_dates=[0], header=None,index_col=0, names=['Fecha','Poblacion'])
df.head(5)

In [ ]:
print(len(df))

In [ ]:
df.describe()

In [ ]:
PASOS=7

# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg
 
# load dataset
values = df.values
# ensure all data is float
values = values.astype('float32')
# normalize features
scaler = MinMaxScaler(feature_range=(-1, 1))
values=values.reshape(-1, 1) # esto lo hacemos porque tenemos 1 sola dimension
scaled = scaler.fit_transform(values)
# frame as supervised learning
reframed = series_to_supervised(scaled, PASOS, 1)
reframed.head()

In [ ]:
# split into train and test sets
values = reframed.values
n_train_days = 120+37 - (30+PASOS)
train = values[:n_train_days, :]
test = values[n_train_days-5:, :]
# split into input and outputs
x_train, y_train = train[:, :-1], train[:, -1]
x_val, y_val = test[:, :-1], test[:, -1]
# reshape input to be 3D [samples, timesteps, features]
x_train = x_train.reshape((x_train.shape[0], 1, x_train.shape[1]))
x_val = x_val.reshape((x_val.shape[0], 1, x_val.shape[1]))
print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [ ]:
def crear_modeloFF():
    model = Sequential() 
    model.add(LSTM(25, return_sequences=True, input_shape=(1,PASOS),activation='tanh'))
    #model.add(Flatten())
    model.add(Dropout(.2))
    #model.add(LSTM(25, return_sequences=True))
    #model.add(Dropout(.2))
    model.add(LSTM(25, return_sequences=False))
    model.add(Dropout(.2))
    model.add(Dense(1, activation='tanh'))
    model.compile(loss='mean_absolute_error',optimizer='Adam',metrics=["mse"])
    model.summary()
    return model

In [ ]:
EPOCHS=5500

model = crear_modeloFF()

history=model.fit(x_train,y_train,epochs=EPOCHS,validation_data=(x_val,y_val),batch_size=PASOS)

# Train & Test

In [ ]:
train_test=model.predict(x_train)

compara = pd.DataFrame(np.array([y_train, [x[0] for x in train_test]])).transpose()
compara.columns = ['x_train', 'y_train']

inverted = scaler.inverse_transform(compara.values).round()

compara2 = pd.DataFrame(inverted)
compara2.columns = ['x_train', 'y_train']
#compara2['diferencia'] = compara2['x_train'] - compara2['y_train']
#compara2.head(10)
print(compara2.shape)

In [ ]:
compara2['x_train'].plot()
compara2['y_train'].plot()
guardar = pd.DataFrame(compara2)
guardar.to_csv('xytrainP.csv', header=True, index=False)

# Validate

In [ ]:
validar=model.predict(x_val)

cmp = pd.DataFrame(np.array([y_val, [x[0] for x in validar]])).transpose()
cmp.columns = ['x_val', 'y_val']

inverted = scaler.inverse_transform(cmp.values).round()

cmp2 = pd.DataFrame(inverted)
cmp2.columns = ['x_val', 'y_val']
#compara2['diferencia'] = compara2['x_train'] - compara2['y_train']
#compara2.head(10)
print(cmp2.shape)

In [ ]:
cmp2['x_val'].plot()
cmp2['y_val'].plot()
guardar2 = pd.DataFrame(cmp2)
guardar2.to_csv('xyvalidateP.csv', header=True, index=False)

# Pronostico

In [ ]:
ultimosDias = df['2020-04-23':'2020-04-30']
ultimosDias

In [ ]:
print(len(ultimosDias))

In [ ]:
values = ultimosDias.values
values = values.astype('float32')
# normalize features
values=values.reshape(-1, 1) # esto lo hacemos porque tenemos 1 sola dimension
scaled = scaler.fit_transform(values)
reframed = series_to_supervised(scaled, PASOS, 1)
reframed.drop(reframed.columns[[7]], axis=1, inplace=True)
reframed.head(7)

In [ ]:
values = reframed.values
x_test = values[6:, :]
x_test = x_test.reshape((x_test.shape[0], 1, x_test.shape[1]))
x_test

In [ ]:
def agregarNuevoValor(x_test,nuevoValor):
    for i in range(x_test.shape[2]-1):
        x_test[0][0][i] = x_test[0][0][i+1]
    x_test[0][0][x_test.shape[2]-1]=nuevoValor
    return x_test

results=[]
for i in range(7):
    parcial=model.predict(x_test)
    results.append(parcial[0])
    print(x_test)
    x_test=agregarNuevoValor(x_test,parcial[0])

In [ ]:
adimen = [x for x in results]    
inverted = scaler.inverse_transform(adimen).round()
inverted

In [ ]:
prediccion1SemanaDiciembre = pd.DataFrame(inverted)
prediccion1SemanaDiciembre.columns = ['pronostico']
prediccion1SemanaDiciembre.plot()


In [ ]:
dfn = pd.DataFrame(prediccion1SemanaDiciembre)

dfn.to_csv('pP.csv', header=True, index=False)